#1. SPD Matrices and $LDL^t$ Factorization

Given an SPD matrix $A$, it can be factorized into $LDL^T$, where $L$ is an unitary lower triangular matrix and $D$ is a diagonal matrix.  To construct the factorization, we follow the Algorithm 6.5 on page 417:
* Do i = 1, ..., n
  * (Step2): For $j = 1,\dots, i-1$, set $v_j = l_{ij}d_j$ 
  * (Step3): Set $d_i = a_{ii} - \displaystyle\sum_{j=1}^{i-1} l_{ij}v_j$ 
  * (Step4): For $j = i+1, \dots, n$, set $l_{ji} = \big( a_{ji} - \displaystyle\sum_{k=1}^{i-1}l_{jk}v_k \big) /d_i$

In [5]:
function LDLt(A)
    #Given an SPD matrix A_{n, n}
    #Output L and D
    n, n1 = size(A)
    v = zeros(n,1)
    L = eye(n)
    d = zeros(n,1)
    for i = 1:n
        # step 2
        for j = 1:i-1
            v[j] = L[i,j] * d[j]
        end
        
        # step 3
        temp = 0
        for j = 1:i-1
            temp = temp + L[i,j] * v[j]
        end
        d[i] = A[i,i] - temp
        
        #step 4
        for j = i+1:n
            temp = 0
            for k = 1:i-1
                temp = temp + L[j,k] * v[k]
            end
            L[j,i] = (A[j,i] - temp) / d[i]
        end
    end
    return L, d
end

LDLt (generic function with 1 method)

Test the code on Example 3 on page 418:
$$A =
\begin{pmatrix}
4 & -1 & 1 \\
-1 & 4.25 & 2.75 \\
1 & 2.75 & 3.5
\end{pmatrix}
,
$$
and its $LDL^t$ factorization is
$$L = 
\begin{pmatrix}
1 & 0 & 0 \\
-0.25 & 1 & 0 \\
0.25 & 0.75 & 1
\end{pmatrix}
\quad
D = 
\begin{pmatrix}
4 & 0 & 0 \\
0 & 4 & 0 \\
0 & 0 & 1
\end{pmatrix}
.
$$

In [6]:
A = [4 -1 1; -1 4.25 2.75; 1 2.75 3.5]
LDLt(A)

(
3x3 Array{Float64,2}:
  1.0   0.0   0.0
 -0.25  1.0   0.0
  0.25  0.75  1.0,

3x1 Array{Float64,2}:
 4.0
 4.0
 1.0)

#2. Tridiagonal Matrix
Tridiagonal matrix can be factorized into $LU$, where $L$ is a lower triangular matrix, $U$ is an unitary upper triangular matrix, and both of them have tridiagonal form.
* $l_{11} = a_{11}$
* $u_{12} = a_{12}/l_{11}$
* Do i = 2, ..., n-1
  * $l_{i, i-1} = a_{i,i-1}$
  * $l_{ii} = a_{ii} - l_{i,i-1} u_{i-1, i}$
  * $u_{i,i+1} = a_{i, i+1}/l_{ii}$
* $l_{n,n-1} = a_{n,n-1}$
* $l_{n,n} = a_{nn} - l_{n,n-1} u_{n-1,n}$

In [10]:
function tridiagLU(A)
    n, n1 = size(A)
    L = zeros(n,n)
    U = eye(n)
    L[1,1] = A[1,1]
    U[1,2] = A[1,2] / L[1,1]
    for i = 2:n-1
        L[i, i-1] = A[i,i-1]
        L[i,i] = A[i,i] - L[i,i-1] * U[i-1,i]
        U[i,i+1] = A[i,i+1] / L[i,i]
    end
    L[n,n-1] = A[n,n-1]
    L[n,n] =  A[n,n] - L[n,n-1]* U[n-1,n]
    return L, U
end

tridiagLU (generic function with 1 method)

Test it on Example 5 on page 423:
$$
T = 
\begin{pmatrix}
2 & -1 & 0 & 0 \\
-1 & 2 & -1& 0 \\
0 & -1 & 2 & -1 \\
0 & 0 & -1 & 2
\end{pmatrix}
$$
and its LU factorization is 
$$
L = 
\begin{pmatrix}
2 & 0 & 0 & 0 \\
-1 & \frac{3}{2} & 0 & 0 \\
0 & -1 & \frac{4}{3} & 0 \\
0 & 0 & -1 & \frac{5}{4}
\end{pmatrix}
,\quad
U = 
\begin{pmatrix}
1 & -\frac{1}{2} & 0 & 0 \\
0 & 1 & -\frac{2}{3} & 0 \\
0 & 0 & 1 & -\frac{3}{4} \\
0 & 0 & 0 & 1
\end{pmatrix}.
$$

In [11]:
T = [2 -1 0 0; -1 2 -1 0; 0 -1 2 -1; 0 0 -1 2]
tridiagLU(T)

(
4x4 Array{Float64,2}:
  2.0   0.0   0.0      0.0 
 -1.0   1.5   0.0      0.0 
  0.0  -1.0   1.33333  0.0 
  0.0   0.0  -1.0      1.25,

4x4 Array{Float64,2}:
 1.0  -0.5   0.0        0.0 
 0.0   1.0  -0.666667   0.0 
 0.0   0.0   1.0       -0.75
 0.0   0.0   0.0        1.0 )

#3. Jacobi's Method
The Jacobi's iterative method is to find the solution of $Ax=b$, and can be written as for given $x^{(0)}$,
$$x^{(k)}_i = \frac{1}{a_{ii}} \big[ \sum_{j=1,\;j\ne i}^{n} (-a_{ij}x^{(k-1)}_{j}) + b_i \big]$$, or equivalently
$$x^{(k)}_i = \frac{1}{a_{ii}} \big[ \sum_{j=1}^{i-1} (-a_{ij}x^{(k-1)}_{j}) + \sum_{j=i+1}^{n} (-a_{ij}x^{(k-1)}_{j}) + b_i \big].$$

> Note that super-script, k, represents the iteration numbers, and sub-script, i, represents the i-th entry of x.

In [16]:
function jacobi(A,b, x0, TOL, Nmax)
    n, n1 = size(A)
    x = zeros(n,1)
    for k = 1:Nmax
        for i = 1:n
            temp1 = 0
            for j = 1:i-1
                temp1 = temp1 + A[i,j] * x0[j]
            end
            
            temp2 = 0
            for j = i+1:n
                temp2 = temp2 + A[i,j] * x0[j]
            end
            
            x[i] = (-temp1 - temp2 + b[i]) /  A[i,i]
        end
        res = norm(x - x0)
        println(k, '\t', res)
        if res < TOL
            return x
        end
        x0[1:n] = x[1:n]
    end
    println("Reach the maximum iterations! Solution might be wrong")
end

jacobi (generic function with 1 method)

Test it on Example 1 on page 451.  The linear system $Ax = b$ is given by
$$
\begin{cases}
 &10x_1 - x_2 + 2x_3 &= 6,\\
 &-x_1 + 11x_2 - x_3 + 3x_4 &= 25,\\
 &2x_1 - x_2 + 10x_3 - x_4 &= -11,\\
 & 3x_2 - x_3 + 8x_4 &=15
\end{cases}
,
$$
and the exact solution is $(1, 2, -1, 1)$.

In [17]:
A1 = [10 -1 2 0; -1 11 -1 3; 2 -1 10 -1; 0 3 -1 8]
b = [6;25;-11;15]
jacobi(A1, b, zeros(4,1), 1e-8, 100)

1	

4x1 Array{Float64,2}:
  1.0
  2.0
 -1.0
  1.0

3.201704898362488
2	1.2556434177591917
3	0.49690551414887585
4	0.2190851701672345
5	0.0897453205646942
6	0.03927457310200085
7	0.01632327649212943
8	0.007087147155725075
9	0.00297279906969194
10	0.0012831613556189962
11	0.0005414124462407633
12	0.0002327443731639058
13	9.857302867807452e-5
14	4.225903498255565e-5
15	1.794098892760319e-5
16	7.677445137551283e-6
17	3.2645209738750824e-6
18	1.395299610258743e-6
19	5.938924510776512e-7
20	2.536369041495742e-7
21	1.0802811616549007e-7
22	4.611225048507366e-8
23	1.9648317139180354e-8
24	8.384120565886267e-9


#4. Gauss-Seidel Method
The GS method can be written as
$$x^{(k)} = \frac{1}{a_{ii}} \big[ \sum_{j=1}^{i-1} (-a_{ij}x^{(k)}_{j}) + \sum_{j=i+1}^{n} (-a_{ij}x^{(k-1)}_{j}) + b_i \big].$$

In [18]:
function GS(A,b, x0, TOL, Nmax)
    n, n1 = size(A)
    x = zeros(n,1)
    for k = 1:Nmax
        for i = 1:n
            temp1 = 0
            for j = 1:i-1
                temp1 = temp1 + A[i,j] * x[j]
            end
            
            temp2 = 0
            for j = i+1:n
                temp2 = temp2 + A[i,j] * x0[j]
            end
            
            x[i] = (-temp1 - temp2 + b[i]) /  A[i,i]
        end
        res = norm(x - x0)
        println(k, '\t', res)
        if res < TOL
            return x
        end
        x0[1:n] = x[1:n]
    end
    println("Reach the maximum iterations! Solution might be wrong")
end

GS (generic function with 1 method)

In [19]:
GS(A1, b, zeros(4,1), 1e-8, 100)

1	

4x1 Array{Float64,2}:
  1.0
  2.0
 -1.0
  1.0

2.742864757228523
2	0.5302971831879821
3	0.044830810568892175
4	0.007109620693563844
5	0.0008743589512862559
6	9.062087893677911e-5
7	8.219392165061605e-6
8	6.539341072741501e-7
9	4.4233701694032134e-8
10	2.3482754714404746e-9
